## Chapter 6 FAMA AND MACBETH REGRESSION ANALYSIS

In Chapter 5, we presented a technique, portfolio analysis, for examining the crosssectional relation between two variables. 

**The major benefit of portfolio analysis** is that it is a nonparametric technique, meaning that it does not make any assumptions
about the nature of the relation between the variables under investigation. 

**The drawback of portfolio analyses** is that it is difficult to include a large set of controls when examining the relation.

In the remainder of this chapter, we present the FM regression technique and exemplify its implementation using the data from our methodology sample. Specifically,we illustrate the FM regression technique using future excess stock returns ($ r_{t+1}$) as the dependent variable, with the stock’s beta ($\beta$), size ($Size$), and book-to-market ratio($BM$) as the independent variables.

**How to do FM regression?**

* The first step is to run periodic cross-sectional regressions of the dependent variable of interest, which we denote $Y$, on one or more independent variables $X_1$, $X_2$, etc., using data from each time period $t$.

* The second step is to analyze the time series of each of the regression coefficients to determine whether the average coefficient differs from zero.

The first step in the FM regression technique is to run a cross-sectional regression ofthe dependent variable $Y$ on the independent variables $X_1, X_2$, etc. In most cases, the cross-sectional regressions will include an intercept term. Thus, our cross-sectional regression specification is:

$$
Y_{i,t}=\delta_{0,t}+\delta_{1,t}X1_{i,t}+\delta_{2,t}X2_{i,t}+…+\epsilon_{i,t}
$$

The independent variables are usually **winsorized** to ensure that a small number of extreme independent variable values do not have a large effect on the results of the regression.In some cases, the dependent variable is also winsorized. When the dependent variable is a security return or excess return, this variable is usually not winsorized.

The result is a time series of intercept and slope coefficients $\delta_{0,t}, \delta_{1,t}, \delta_{2,t}$ etc. Each time period will also produce regression statistics such as the R-squared, adjusted R-squared, and number of observations used in the regression. We denote these values from the cross-sectional regression for period t as $R_t^2 , Adj.R^2_t$, and $n_t$, respectively.

Before proceeding to an example, it is worth mentioning that the type of cross-sectional regression used when implementing the FM regression procedure **need not be a standard ordinary-least-squares (OLS) regression**. It is straightforward to replace the OLS regression with a weighted-least-squares regression or even a logistic regression or probit model if the dependent variable is binary. Multinomial models are also possible. The procedure is therefore quite flexible and can be applied to examine a wide array of economic phenomena.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats.mstats import winsorize

In [2]:
#导入数据
with open('alldata2021.csv') as file:
    alldata=pd.read_csv(file)
#删除2012年数据
alldata=alldata[alldata['year']!=2012]
alldata.head()

,PERMNO,year,beta,BM,rt_1,MktCap,Size
0,10001,1988,0.255128,1.139535,0.604226,6.362250,1.850382
1,10002,1988,0.025613,NaN,-0.352357,9.840625,2.286519
2,10003,1988,0.280791,NaN,-0.616695,41.686000,3.730165
3,10005,1988,0.480927,1.250486,-0.417034,0.785250,-0.241753
4,10006,1988,NaN,NaN,NaN,NaN,NaN


In [3]:
#原代码
def FM_regression1(independent, level=0.005):
    coefs = []
    R_square = []
    adj_R = []
    number = []
    # 筛选出所需指标数据
    FM_df = df[(['year', 'rt+1'] + independent)].copy()
    for i in range(df['year'].min(), df['year'].max()):  # 不能取2012年数据
        temp = FM_df[FM_df['year'] == i].copy()
        temp = temp.dropna()  # 剔除缺失值
        number.append(len(temp))  # 样本量
        temp = temp.drop(columns='year')
        temp[independent] = temp[independent].apply(
            lambda x: x.clip(
                np.percentile(x, level * 100),
                np.percentile(x, (1 - level) * 100)
            )
        )  # 自变量缩尾
        Y = temp['rt+1']  # 因变量
        X = temp[independent]  # 自变量
        model = sm.OLS(Y.values, sm.add_constant(X).values).fit()
        #number.append(model.nobs)
        coefs.append(model.params)
        R_square.append(model.rsquared)
        adj_R.append(model.rsquared_adj)
    # 按课本Tabel 6.1
    result = pd.DataFrame(
        coefs,
        index=range(df['year'].min(), df['year'].max()),
        columns=['coef' + str(j) for j in range(len(independent) + 1)]
    )
    result['R_square'] = R_square
    result['adj_R'] = adj_R
    result['n'] = number

    return result

In [4]:
def FM_regression(df,dv=['rt_1'],idv=['beta'],level=0.005):
    def regression(dff,dv=dv,idv=idv,level=level):
        #缩尾winsorize()
        Y,X=dff[dv],dff[idv].apply(lambda x:winsorize(x,[level,level]).data)
        model=sm.OLS(Y.values,sm.add_constant(X).values).fit()
        res=pd.DataFrame(np.array([model.params[i]*100 for i in range(len(idv)+1)]+[model.rsquared,model.rsquared_adj,len(Y)]).reshape(1,-1),columns=['coef%s'%i for i in range(len(idv)+1)]+['R2','R2_adj','n'],index=[0])
        #保留小数round()
        return  res.round(pd.Series([2]*(len(res.columns)-3)+[3,3,0],index=res.columns))
    #剔除缺失值dropna()
    res=df[['PERMNO','year']+dv+idv].dropna().groupby('year').apply(regression)
    res.index=res.index.droplevel(1)
    return res

In [5]:
panelA=FM_regression(alldata,dv=['rt_1'],idv=['beta'],level=0.005)
panelA

,coef0,coef1,R2,R2_adj,n
year,,,,,
1988,2.75,4.90,0.001,0.001,6031.0
1989,-29.43,2.97,0.001,0.001,5843.0
1990,40.88,11.38,0.002,0.002,5652.0
1991,36.47,-17.64,0.008,0.008,5581.0
1992,28.52,-9.70,0.010,0.010,5721.0
1993,-4.76,-0.74,0.000,-0.000,5961.0
1994,25.81,2.66,0.000,0.000,6495.0
1995,19.41,-6.44,0.005,0.005,6667.0
1996,31.83,-17.60,0.024,0.024,6943.0


![title](fig/fig0.png)

In [6]:
panelB=FM_regression(alldata,dv=['rt_1'],idv=['Size'],level=0.005)
panelB

,coef0,coef1,R2,R2_adj,n
year,,,,,
1988,-7.45,3.28,0.012,0.012,6394.0
1989,-33.59,1.39,0.004,0.004,6192.0
1990,66.40,-5.33,0.007,0.007,5978.0
1991,53.35,-7.14,0.020,0.020,6051.0
1992,45.31,-5.45,0.021,0.021,6242.0
1993,-3.08,-0.71,0.001,0.001,6734.0
1994,31.08,-0.69,0.000,0.000,7088.0
1995,20.15,-1.35,0.001,0.001,7355.0
1996,14.74,0.75,0.000,0.000,7821.0


![title](fig/fig1.png)
![title](fig/fig2.png)

In [7]:
panelC=FM_regression(alldata,dv=['rt_1'],idv=['BM'],level=0.005)
panelC

,coef0,coef1,R2,R2_adj,n
year,,,,,
1988,2.27,4.35,0.003,0.003,4997.0
1989,-26.17,0.25,0.000,-0.000,4860.0
1990,55.36,-6.09,0.002,0.002,4770.0
1991,12.55,8.18,0.007,0.007,4884.0
1992,17.56,3.84,0.002,0.002,5078.0
1993,-12.34,7.30,0.010,0.010,5522.0
1994,26.22,0.05,0.000,-0.000,5891.0
1995,5.71,10.70,0.009,0.009,6118.0
1996,4.83,11.57,0.012,0.012,6550.0


![title](fig/fig3.png)
![title](fig/fig4.png)

In [8]:
panelD=FM_regression(alldata,dv=['rt_1'],idv=['beta','Size','BM'],level=0.005)
panelD

,coef0,coef1,coef2,coef3,R2,R2_adj,n
year,,,,,,,
1988,-10.12,-1.89,3.64,4.14,0.016,0.015,4697.0
1989,-31.16,-1.08,1.43,0.45,0.004,0.003,4572.0
1990,77.65,24.38,-10.10,-6.85,0.023,0.022,4506.0
1991,43.51,-6.29,-5.97,5.68,0.023,0.022,4455.0
1992,44.11,-5.43,-4.67,1.27,0.023,0.022,4642.0
1993,-8.87,2.08,-0.80,7.30,0.011,0.010,4899.0
1994,29.39,4.65,-1.59,0.40,0.002,0.001,5412.0
1995,19.70,-5.22,-1.44,7.16,0.012,0.012,5546.0
1996,7.12,-12.03,2.16,7.82,0.022,0.021,5803.0


![title](fig/fig5.png)

## 6.2 Summarized FM Regression Results

In [9]:
panelA.head(10)

,coef0,coef1,R2,R2_adj,n
year,,,,,
1988,2.75,4.90,0.001,0.001,6031.0
1989,-29.43,2.97,0.001,0.001,5843.0
1990,40.88,11.38,0.002,0.002,5652.0
1991,36.47,-17.64,0.008,0.008,5581.0
1992,28.52,-9.70,0.010,0.010,5721.0
1993,-4.76,-0.74,0.000,-0.000,5961.0
1994,25.81,2.66,0.000,0.000,6495.0
1995,19.41,-6.44,0.005,0.005,6667.0
1996,31.83,-17.60,0.024,0.024,6943.0


**Standard errors,t-statistics, and p-values are calculated using the Newey and West (1987) adjustment with six lags.**

In [11]:
def NWtest(a, lags=6):
    '''
    一个序列的NW检验

    Parameters
    ----------
    a: 需要检验的序列  (array-like)
    lags: NW检验的最大滞后阶数  (float)

    Returns
    -------
    序列均值  (float)
    NW调整后标准误  (float)
    NW调整后标准误  (float)
    p值  (float)
    '''
    adj_a = np.array(a)
    # 对常数序列回归
    model = sm.OLS(adj_a, [1] * len(adj_a)).fit(cov_type='HAC', cov_kwds={'maxlags': lags})

    return adj_a.mean(), float(np.sqrt(model.cov_params())), float(model.tvalues), float(model.pvalues)

def TS_summary(data, name, **kwds):
    '''
    将table 6.1中的一个panel转化成6.2中一列的形式并保存

    Parameters
    ----------
    data: 6.1中一个panel样式的数据  (pd.DaFrame)
    name: 结果名，含扩展名  (str)
    '''
    # 系数的NW检验
    result1 = data.iloc[:, :-3].apply(NWtest, **kwds)
    result1 = np.array([list(x) for x in result1.values]).reshape(-1,order='F')  # 转化成一维
    result1 = result1.round(2)
    
    # 最后三列求平均值即可
    result2 = data.iloc[:, -3:].mean().values.round(3)
    result2[2]=int(result2[2])
    
    # 转成df保存到csv
    if int(name) < 4:
        result_index=['Average(c0)','Standard error(c0)','t-statistic(c0)','p-value(c0)',
               'Average(c'+name+')','Standard error(c'+name+')','t-statistic(c'+name+')','p-value(c'+name+')',
              'R2','Adj.R2','n']   
        result = pd.DataFrame({'Coefficient Value': result_index,'('+name+')': list(result1) + list(result2)})
    else:
        result = pd.DataFrame(list(result1) + list(result2))
    name1='column'+str(name)
    result.to_csv(name1,index=False)

In [12]:
# 四个panel全部检验
[TS_summary(i, j) for i, j in zip([panelA, panelB, panelC,panelD], ['1', '2', '3','4'])]

[None, None, None, None]

In [14]:
# 面板结果汇总与可视化
df=pd.read_csv('column1')
for i in range(2,4):
    name='column'+str(i)
    a=pd.read_csv(name)
    df=df.merge(a,on='Coefficient Value',how='outer')

a=df[df['Coefficient Value'].isin(['R2','Adj.R2','n'])]
df1=df[~df['Coefficient Value'].isin(['R2','Adj.R2','n'])]
df1=pd.concat([df1,a])

d=pd.read_csv('column4')
df1['(4)']=d.values
df1.fillna(value='',inplace=True)
df1=df1.reset_index()
del df1['index']
df1

,Coefficient Value,(1),(2),(3),(4)
0,Average(c0),15.04,23.37,10.03,22.870
1,Standard error(c0),2.88,4.79,2.14,4.820
2,t-statistic(c0),5.23,4.88,4.68,4.750
3,p-value(c0),0,0,0,0.000
4,Average(c1),-2.65,,,1.020
5,Standard error(c1),1.87,,,1.750
6,t-statistic(c1),-1.42,,,0.580
7,p-value(c1),0.16,,,0.560
8,Average(c2),,-2.26,,-2.580
9,Standard error(c2),,0.59,,0.610


The table shows that, for example, using the specification that includes 𝛽, Size, and BM as independent variables (specification (4)), the average value of the intercept coefficient (𝛿0) is 22.87(21.74) and its standard error is 4.82(4.58), giving a t-statistic of 4.75(4.75) and a p-value of very close to zero. The average coefficient on 𝛽 (𝛿1) is 1.02(0.96) with a standard error of 1.75(1.68), t-statistic of 0.58(0.57), and p-value of 0.56(0.57).

![title](fig/fig6.png)

The average R-squared and adjusted R-squared values from the regressions that include all three variables are 0.025 and 0.024, respectively, indicating that only a little more than 2% of the total cross-sectional variation in future stock returns is explained by 𝛽, Size, and BM. Low levels of R-squared such as these are quite common in research that examines the ability to predict future stock returns.

In summary, our analysis finds statistically and economically important relations between expected returns and each of Size and BM, with Size being negatively related to expected returns and BM being positively related. The results indicate no relation between 𝛽 and expected stock returns. These results are consistent with the results we found using portfolio analysis in Chapter 5.

## 6.3 PRESENTING FM REGRESSIONS

In most cases, only one inferential statistic is presented. Thus, instead of presenting the standard errors, t-statistics, and p-values,
only one of these is presented. In most cases, researchers choose to present t-statistics, and usually the t-statistics are presented in parentheses to alleviate the need for the column labeled Value in Table 6.2.

In [15]:
# 表格简化输出
table3_sub_index = ['Intercept','','beta','','size','','BM','','Adj.R2','n']
table3_sub_columns = ['(1)','(2)','(3)','(4)']
table3_sub = pd.DataFrame(index=table3_sub_index,columns=table3_sub_columns)

for i in range(len(table3_sub_columns)):
    
    for j in range(len(table3_sub_index)-2):
        
        if j % 2 ==0:
            table3_sub.iloc[j,i] = df1.iloc[2*j,i+1]
        else:
            table3_sub.iloc[j,i] = '(' + str(df1.iloc[2*j,i+1]) + ')'
    
    table3_sub.iloc[len(table3_sub_index)-2,i] = df1.iloc[-2,i+1]
    table3_sub.iloc[len(table3_sub_index)-1,i] = df1.iloc[-1,i+1]
    
table3_sub[table3_sub.values=='()']=''
table3_sub

,(1),(2),(3),(4)
Intercept,15.04,23.37,10.03,22.87
,(5.23),(4.88),(4.68),(4.75)
beta,-2.65,,,1.02
,(-1.42),,,(0.58)
size,,-2.26,,-2.58
,,(-3.79),,(-4.24)
BM,,,3.97,2.63
,,,(5.5),(5.5)
Adj.R2,0.011,0.01,0.006,0.024
n,5555,5937,4874,4549


![title](fig/fig7.png)

In [ ]:
pip install linearmodels

In [30]:
# 使用python自带的FM函数实现
from linearmodels import FamaMacBeth

fmdata = alldata.set_index(['PERMNO','year'])
fmdata.iloc[:,1:]=fmdata.iloc[:,1:].apply(lambda x:winsorize(x,[0.005,0.005]).data)
fmdata.dropna(inplace=True)

fm = FamaMacBeth(dependent = fmdata['rt_1'],exog = sm.add_constant(fmdata[['beta','BM','Size']])) # exog为自变量
res_fm = fm.fit(cov_type= 'kernel',debiased = False,bandwidth = 6) # kernel表示启用NW调整，bandwidth为最大滞后阶数
res_fm

Dep. Variable:,rt_1,R-squared:,0.0058
Estimator:,FamaMacBeth,R-squared (Between):,-0.0785
No. Observations:,109184,R-squared (Within):,0.0146
Date:,"Tue, Jun 22 2021",R-squared (Overall):,0.0058
Time:,21:11:39,Log-likelihood,-1.548e+05
Cov. Estimator:,Fama-MacBeth Kernel Cov,,
,,F-statistic:,212.88
Entities:,20449,P-value,0.0000
Avg Obs:,5.3393,Distribution:,"F(3,109180)"
Min Obs:,0.0000,,
Max Obs:,74.000,F-statistic (robust):,32.105


## 6.4 SUMMARY

FM regression analysis is used to examine the cross-sectional relation between a dependent variable and one or more independent
variables in the average time period. 

The main benefit of FM regression analysis is that it allows us to control for a large set of potential explanations for the phenomenon under investigation. The regression can eliminate the influence of cross-section correlation on standard error.

The drawback is that it requires assumptions regarding the nature of the relation between the dependent and independent variables. In
most cases, this relation is assumed to be linear, in which case OLS regression (or potentially weighted-least-squares regression) is used to perform the periodic cross-sectional analysis.